# 2. Extracting Token Transfers
In this excercise, we will be extracting token transfers from a single token contract.
We will obtain these transfers from the event/log data, using web3py.

As in the previous excercise, please use your own endpoint URL. You can create a free account with [Moralis.io](https://admin.moralis.io/register), and get an endpoint URL from them. You may be able to use this notebook endpoint, but you may run into rate limits if other participants are using it at the same time. Or at a later point in time this endpoint may not work at all anymore.

In [1]:
endpoint = "https://speedy-nodes-nyc.moralis.io/03c966587b022c980f59136b/eth/mainnet/archive"

In [20]:
from web3 import Web3
w3 = Web3(Web3.HTTPProvider(endpoint))
w3.isConnected()

True

# 2.1 Defining the token contract address and extracting logs
We can now specify a token contract address. For illustration purposes, we'll use the Bionic token contract address. (Feel free to change it!)

Using the Moralis.io endpoint, we then extract logs in a certain block range, but in batches of 2000 blocks. This is the maximum range that the Moralis.io endpoint allows. If you run your own node, you could use larger intervals.

The result will be event logs in JSON format, but not parsed yet. So transfer amounts would still be in hexadecimal format for example.

In [22]:
tokenContractAddress = "0xef51c9377feb29856e61625caf9390bd0b67ea18" # Bionic token contract address

In [51]:
from tqdm.notebook import tqdm
import itertools

logLists = []
blockStart = 6000000
blockEnd = blockStart + 300000

for blockNumber in tqdm(range(blockStart, blockEnd, 2000)):
    logs = w3.eth.get_logs({"fromBlock": str(hex(blockNumber)),
                            "toBlock": str(hex(blockNumber+2000)),
                            "address": Web3.toChecksumAddress(tokenContractAddress),
                            "topics": [Web3.keccak(text='Transfer(address,address,uint256)').hex()]})
    logLists.append(logs)
logs = list(itertools.chain.from_iterable(logLists))

  0%|          | 0/150 [00:00<?, ?it/s]

# 2.2 Parsing JSON logs with the Transfer event ABI
While smart contracts written in high level languages like Solidity are compiled to EVM bytecode, accessing the bytecode functionality through function names would still be very useful. This is where the Application Binary Interface (ABI) comes in. It provides information on function and event signatures, which allows for a translation to bytecode entrypoints. EVM smart contract developers usually generate this ABI for their code. [You can obtain such ABIs from Etherscan (at the bottom of the page)](https://etherscan.io/address/0xef51c9377feb29856e61625caf9390bd0b67ea18#code), where developers frequently upload them. If you want to learn more, [quicknode has a good article on ABIs](https://www.quicknode.com/guides/solidity/what-is-an-abi)

Using the ABI of a transfer event let's us easily parse event logs that conform to the token transfer signature.
During parsing, we create a custom dictionary format because web3py's internal structure is nested and immutable.

In [52]:
import json
# Reduced ERC-20 ABI, only Transfer event
ABIstring = """[ { "anonymous": false, "inputs": [
            {   "indexed": true,
                "name": "from",
                "type": "address"
            },
            {   "indexed": true,
                "name": "to",
                "type": "address"
            },
            {   "indexed": false,
                "name": "value",
                "type": "uint256"
            } ], "name": "Transfer", "type": "event" } ]"""
anonERC20contract = w3.eth.contract(abi=json.loads(ABIstring))
transferEventType = anonERC20contract.events.Transfer
transferEventABI = transferEventType._get_event_abi()

In [53]:
# With the ABI, we can now parse the transfer event logs and create our custom, unnested format.
from web3._utils.events import get_event_data
transferEvents = []
for log in logs:
        log = dict(get_event_data(w3.codec, transferEventABI, log))
        log["transactionHash"] = log["transactionHash"].hex()
        del log["blockHash"]
        for k,v in log["args"].items():
            log[k] = v
        del log["args"]
        transferEvents.append(log)

# 2.3 Creating a pandas dataframe
At this point we can transform the transfer event JSON list into a pandas dataframe!

In [54]:
import pandas as pd
tokenTransfersDF = pd.DataFrame(transferEvents)
tokenTransfersDF

,event,logIndex,transactionIndex,transactionHash,address,blockNumber,from,to,value
0,Transfer,44,95,0x4f3f96adef9ca0444209b4844dea4049f97a74045d45...,0xEf51c9377FeB29856E61625cAf9390bD0B67eA18,6009316,0xdD2A5B646bb936CbC279CBE462E31eab2C309452,0x24cdEABeD51cCacD3199b410236Cd7E24ca1d313,2500000000000
1,Transfer,17,68,0xaa9369523e8aeff77c457bcc2bfbdc6f17fc0be16794...,0xEf51c9377FeB29856E61625cAf9390bD0B67eA18,6010109,0xdD2A5B646bb936CbC279CBE462E31eab2C309452,0xFe0AedEe5FaC7037b7DEc21F05a58c954df5A0F0,2500000000000
2,Transfer,20,29,0xfec0cb27cc608809998b6fb6ab35e5540b38de5676de...,0xEf51c9377FeB29856E61625cAf9390bD0B67eA18,6010118,0xdD2A5B646bb936CbC279CBE462E31eab2C309452,0x817F0fc8760Eac5c497697329b50f632c453a4eB,2500000000000
3,Transfer,46,72,0x7534c86ba66f6636c94ebf9cc46db22b77d9c70c9bc3...,0xEf51c9377FeB29856E61625cAf9390bD0B67eA18,6010123,0xdD2A5B646bb936CbC279CBE462E31eab2C309452,0x289BC64F25Af2785669DA79126a7B26A76cA956e,2500000000000
4,Transfer,44,85,0x2fb9ba27f354dfa73a3055f559bef749748265f400e4...,0xEf51c9377FeB29856E61625cAf9390bD0B67eA18,6010127,0xdD2A5B646bb936CbC279CBE462E31eab2C309452,0xC802506207A588cF85191B14693743d3777aDd8B,2500000000000
...,...,...,...,...,...,...,...,...,...
16925,Transfer,28,23,0x90f2ff8d20256d699e692f796ad49a51364250d6fa79...,0xEf51c9377FeB29856E61625cAf9390bD0B67eA18,6299508,0x4E33d8c2CA8c74FaD0905E42C7f3317019774f50,0x274F3c32C90517975e29Dfc209a23f315c1e5Fc7,115266668157343
16926,Transfer,11,18,0x04797f21607e80a5472867c7ea98cb4aa9021615fdff...,0xEf51c9377FeB29856E61625cAf9390bD0B67eA18,6299640,0xBb02AB3275D79007c0acAf381d36F6Bd78abeCbd,0x274F3c32C90517975e29Dfc209a23f315c1e5Fc7,2500000000000
16927,Transfer,20,50,0xfd4a1623f1f069c0525d95939c6903da33c66776e8bb...,0xEf51c9377FeB29856E61625cAf9390bD0B67eA18,6299689,0x233F0dd15867FabC5195deb251eB68432FcafD98,0x2a0c0DBEcC7E4D658f48E01e3fA353F44050c208,2500000000000
16928,Transfer,14,36,0x39894fc4393d779789c6597b48e6f1e0f9590d2f22bb...,0xEf51c9377FeB29856E61625cAf9390bD0B67eA18,6299823,0x1138BCa344eEec9c8e319Dbb54e05FCe726D1e3B,0x274F3c32C90517975e29Dfc209a23f315c1e5Fc7,2500000000000


# 2.4 Who is distributing tokens with the same value (airdrop)?
From previous knowledge about this token network we know that there was an airdrop.

Which account has airdropped the tokens? We can simply group by from address and value, and count the number of transfers:

In [93]:
senders = tokenTransfersDF.groupby(['from','value']).agg({'to':'count'}).sort_values("to", ascending=False)
senders.reset_index(inplace=True)
senders.head()

,from,value,to
0,0xdD2A5B646bb936CbC279CBE462E31eab2C309452,2500000000000,9954
1,0xdD2A5B646bb936CbC279CBE462E31eab2C309452,100000000000,27
2,0xdD2A5B646bb936CbC279CBE462E31eab2C309452,1000000000000000,10
3,0x8d12A197cB00D4747a1fe03395095ce2A5CC6819,2500000000000,9
4,0x3c8bB860d09c8E50a4E85331F66E418d071Bf080,35000000000000,4


# 2.5 Who is receiving tokens of the same value multiple times?
We do the same in reverse, we group by receiving address and value, and count the number of transfers:

In [90]:
receivers = tokenTransfersDF.groupby(['to','value']).agg({'from':'count'}).sort_values("from", ascending=False)
receivers.reset_index(inplace=True)
receivers.head()

,to,value,from
0,0xE6c2d451936dFCA11fa968426b93E70F8A135221,2500000000000,1072
1,0xF21329C8A24a19388e8eb91A7c710D6fa10dEDE6,2500000000000,688
2,0xC4fd7c26aE028BF42F60FA6a0eF5c32990fCdA9f,2500000000000,445
3,0x715D4B5180fd31Ab9ABaf63646c6AFdb3Ce37a3C,2500000000000,411
4,0xa02E73A0564874Cd17B82669E72daE170AcF0371,2500000000000,315


# 2.6 Create a token transfer graph with networkx

In [89]:
import networkx as nx
G = nx.from_pandas_edgelist(df=tokenTransfersDF,
                            source="from", target="to",
                            create_using=nx.DiGraph)

In [62]:
nodeMeasures = pd.DataFrame(dict(
    indegree = dict(G.in_degree),
    outdegree = dict(G.out_degree),
    indegree_centrality = nx.in_degree_centrality(G)
))
nodeMeasures[nodeMeasures.values >= 10]

,indegree,outdegree,indegree_centrality
0xdD2A5B646bb936CbC279CBE462E31eab2C309452,2,9828,0.000195
0x97126cbde15c4582cB5A76dEB1eDD1577C279F69,25,2,0.002443
0x79Af9E6fb38D10D6B99901faa25472D8f53ca121,13,1,0.001271
0xC128e3E58d9b126D2c0fA0E52FA33ea14aa6B83a,31,2,0.003030
0x39c9a982CfE5adB77c4B855309Dc87c12d9db4cC,29,1,0.002834
...,...,...,...
0xF21329C8A24a19388e8eb91A7c710D6fa10dEDE6,694,2,0.067826
0xcbd2A3423ea7095e2C4b650eD4539cc2936045dA,11,1,0.001075
0xcaF30aEf51f28FdAb9e0c6dD77299b67DFCa823B,39,2,0.003812
0x948E0e69De1d1ee9B129cDC3e61a36e7B68408d2,11,1,0.001075


In [102]:
airdropNode = senders.at[0, "from"]
multiReceiverNode = receivers.at[4, "to"] # pick a node that received many same value transfers, but not the top one
incomingNodes = [source for (source, target) in G.in_edges(multiReceiverNode)]
selectedNodes = [airdropNode] + incomingNodes + [multiReceiverNode]

In [104]:
subG = G.subgraph(nodes=selectedNodes)
subG.number_of_edgesdges()

637

In [110]:
nt = Network(height='750px', width='100%', notebook=True)
# populates the nodes and edges data structures
nt.from_nx(subG)
nt.force_atlas_2based()
nt.show('nx.html')

In [109]:
nx.draw(subG)

ImportError: Matplotlib required for draw()

In [106]:
from pyvis.network import Network

In [19]:
transferNetwork = Network(height='750px', width='100%', notebook=True)
# set the physics layout of the network
transferNetwork.barnes_hut()

sampleSize = 500
for row in tokenTransfersDF.sample(sampleSize)[["from","to"]].to_dict('records'):
    transferNetwork.add_node(row["from"], row["from"], title=row["from"])
    transferNetwork.add_node(row["to"], row["to"], title=row["to"])
    transferNetwork.add_edge(row["from"], row["to"], value=1)

In [20]:
transferNetwork.show("tokenTransferGraph.html")

In [13]:
0xef51c9377feb29856e61625caf9390bd0b67ea18 # bionic

1366272683043014078604117687716855698063859640856